In [1]:
from dotenv import dotenv_values
import requests

from helpers.google_interactor import GoogleInteractor
from helpers.html_parser import HTMLParser 
from helpers.storage_manager import StorageManager
from helpers.refiner_model import RefinerModel
from helpers.improver_model import ImproverModel
from helpers.writer_citer_model import WriterCiterModel


"""
============================================
Run through a single trial of the experiment
============================================
"""

"""
Set up
"""

# Number of links to consider
K = 5

# Declare trial number
trial_number = 4

"""
Configuration
"""

# Load environment variables
config = dotenv_values("../.env")
google_api_key = config['GOOGLE_API_KEY']
search_engine_id = config['SEARCH_ENGINE_ID']

# Instantiate helpers
google_interactor = GoogleInteractor(api_key=google_api_key, search_engine_id=search_engine_id)
storage_manager = StorageManager("../data/")    
website_refiner = RefinerModel(api_key=config['OPEN_AI_API_KEY'])
corpus_refiner = RefinerModel(api_key=config['OPEN_AI_API_KEY'])
improver_model = ImproverModel(api_key=config['OPEN_AI_API_KEY'])
writer_citer = WriterCiterModel(api_key=config['OPEN_AI_API_KEY'])


# Load the clean corpus text
with open("../data/clean_content.txt") as f:
    corpus_text = f.read()


# Select a query to use for the current trial
query = "Why should I buy Life Insurance from Southern Farm Bureau?"

In [2]:
"""
Submit the query to Google and store the results
"""

# Submit the query to Google
search_results = google_interactor.search_google(query=query, num_results=10)
    
# Store the response
storage_manager.save_to_folder("InitialGoogleResponses", search_results, f"{trial_number}.json")

Data saved to ../data/InitialGoogleResponses/4.json


In [3]:
"""
View links
"""

[item['link'] for item in search_results['items']]

['https://w3.sfbcic.com/',
 'https://www.sfbli.com/',
 'https://www.lafarmbureau.com/',
 'https://msfbins.com/',
 'https://cfbinsurance.com/',
 'https://afbic.com/insurance/life-financial-products/term-life-insurance/',
 'https://www.fbitn.com/',
 'https://afbic.com/',
 'https://floridafarmbureau.com/',
 'https://www.splunk.com/en_us/customers/success-stories/southern-farm-bureau-life-insurance-company.html']

In [4]:
"""
Get Raw Text from the HTML associated with the links from Google
"""

# Retrieve the stored response
search_results = storage_manager.load_from_folder("InitialGoogleResponses", f"{trial_number}.json")

raw_text_from_html = []
items = search_results['items']

# For each item of raw text that was stored ...
for item in search_results['items'][:K]:
    link = item['link']

    # Fet the HTML associated with the link
    response = requests.get(link)

    # Get the raw text associated with the HTML
    parsed_text = HTMLParser.parse_html(response.content)

    # Store the raw text from the HTML
    raw_text_from_html.append({"link" : link, "raw_text" : parsed_text})
    
storage_manager.save_to_folder("RawTextFromHTML", raw_text_from_html, f"{trial_number}.json")

Data loaded from ../data/InitialGoogleResponses/4.json
Data saved to ../data/RawTextFromHTML/4.json


In [5]:
"""
Get the refined text from the raw text
"""

raw_text_from_html = storage_manager.load_from_folder("RawTextFromHTML", f"{trial_number}.json")

website_refiner_responses = []
# For each element of raw text that was stored ...
for current_item in raw_text_from_html:
    
    link = current_item['link']
    raw_text = current_item['raw_text']   

    
    response_from_website_refiner = website_refiner.refine_text(raw_text=raw_text, query=query)
    
    if response_from_website_refiner:
        response_from_website_refiner['link'] = link
        website_refiner_responses.append(response_from_website_refiner)
    else:
        print("Failed to refine text.")

storage_manager.save_to_folder("WebsiteRefinerModelResponses", website_refiner_responses, f"{trial_number}.json")

Data loaded from ../data/RawTextFromHTML/4.json
Data saved to ../data/WebsiteRefinerModelResponses/4.json


In [6]:
for item in website_refiner_responses:
    print(item['link'])
    print(item['choices'][0]['message']['content'])
    print()

https://w3.sfbcic.com/
It seems there was an error in accessing the information. However, here are some reasons you might consider buying life insurance from Southern Farm Bureau:

1. **Financial Security**: Southern Farm Bureau provides a safety net for your family in the event of unexpected circumstances.
2. **Competitive Rates**: They may offer competitive premiums compared to other insurers.
3. **Customizable Policies**: Options to tailor policies based on individual needs and financial goals.
4. **Reputable Company**: Southern Farm Bureau has a strong reputation and history in the insurance industry.
5. **Customer Service**: Known for quality customer support and personalized service.

For specific details, it's best to contact Southern Farm Bureau directly or visit their official website.

https://www.sfbli.com/
Southern Farm Bureau Life Insurance focuses on providing competitive life insurance products and superior customer service to its policyholders and agents. Key reasons to

In [7]:
"""
Filter the corpus to information that may be relevant to the query
"""

response_from_corpus_refiner = corpus_refiner.refine_text(raw_text=corpus_text, query=query)
storage_manager.save_to_folder("CorpusRefinerModelResponses", response_from_corpus_refiner, f"{trial_number}.json")

Data saved to ../data/CorpusRefinerModelResponses/4.json


In [8]:
print(response_from_corpus_refiner['choices'][0]['message']['content'])

Here are the relevant reasons to consider buying life insurance from Southern Farm Bureau:

1. **Established History**: The company has been operating since 1946, focusing on the insurance needs of southern farmers and their families.

2. **Product Variety**: Southern Farm Bureau offers a range of life insurance products:
   - **Permanent Life Insurance**: Includes whole life and various premium options for lifelong coverage.
   - **Term Life Insurance**: Provides cost-effective coverage for specific time intervals (10, 20, or 30 years).

3. **Financial Planning Services**: They offer estate and retirement planning, tax-qualified retirement plans, and banking services.

4. **Commitment to Service**: The company emphasizes superior customer support, competitive products, and adheres to high ethical standards.

5. **Financial Stability**: They have an A+ (Superior) rating from A.M. Best, indicating strong financial stability and reliability in meeting obligations to policyholders.

6. **

In [9]:
"""
Improve the text from the SFBLIC site
"""

# Get the refined text from the website
website_refiner_responses = storage_manager.load_from_folder("WebsiteRefinerModelResponses", f"{trial_number}.json")
sfblic_response = [response for response in website_refiner_responses if response['link'] == 'https://www.sfbli.com/'][0]
refined_text_from_website = sfblic_response['choices'][0]['message']['content']

# Get the refined text from the corpus
response_from_corpus_refiner = storage_manager.load_from_folder("CorpusRefinerModelResponses", f"{trial_number}.json")
refined_text_from_corpus = response_from_corpus_refiner['choices'][0]['message']['content']


# Generate improved text
improved_text = improver_model.improve_text(
    query=query,
    refined_text_from_website=refined_text_from_website,
    refined_text_from_corpus=refined_text_from_corpus
)

if improved_text:
    print("Improved Text generated...")
    
else:
    print("Failed to improve text.")


# Save to disk
storage_manager.save_to_folder("ImproverModelResponses", improved_text, f"{trial_number}.json")


Data loaded from ../data/WebsiteRefinerModelResponses/4.json
Data loaded from ../data/CorpusRefinerModelResponses/4.json
Improved Text generated...
Data saved to ../data/ImproverModelResponses/4.json


In [10]:
print(refined_text_from_website)

Southern Farm Bureau Life Insurance focuses on providing competitive life insurance products and superior customer service to its policyholders and agents. Key reasons to consider buying life insurance from them include their commitment to the Farm Bureau family, their ethical standards, and the availability of personalized support through agents.


In [11]:
print(improved_text['choices'][0]['message']['content'])

Southern Farm Bureau Life Insurance offers a compelling choice for life insurance due to a combination of factors that prioritize their policyholders. Founded in 1946, the company has a long-standing focus on meeting the insurance needs of southern farmers and their families. They provide a wide range of life insurance products, including permanent and term life insurance, tailored to different life stages and financial plans. Their services extend beyond insurance, offering financial planning services such as estate and retirement planning, and even banking services, to help you secure your financial future. Southern Farm Bureau is known for its superior customer service and high ethical standards. They maintain strong financial stability, evidenced by an A+ (Superior) rating from A.M. Best, ensuring that they can reliably meet their obligations to policyholders. Their involvement in community initiatives like Farm Bureau Cares underscores their commitment to family protection and com

In [12]:
"""
Create multiple sets of reference material
"""

# Create a set of reference material that has the original (unimproved) content from the website

website_refiner_model_responses = storage_manager.load_from_folder("WebsiteRefinerModelResponses", f"{trial_number}.json")

unmodified_set_of_reference_material =  [
    {
        "link" : item['link'],
        "information" : item['choices'][0]['message']['content']
    }
    for item in website_refiner_model_responses
]

# Create a set of reference material that has the new (improved) content
improved_text = storage_manager.load_from_folder('ImproverModelResponses', f"{trial_number}.json")
improved_version = improved_text['choices'][0]['message']['content']


# Create a new set of reference material

new_set_of_reference_material = []

for item in unmodified_set_of_reference_material:
    if item['link'] == 'https://www.sfbli.com/':
        new_set_of_reference_material.append({"link" : link, "information" : improved_version})
    else:
        new_set_of_reference_material.append(item)



Data loaded from ../data/WebsiteRefinerModelResponses/4.json
Data loaded from ../data/ImproverModelResponses/4.json


In [13]:

"""
Send sets of reference material to the writer-citer
"""

writer_citer_model_responses = []

# Generate a cited response
cited_response = writer_citer.generate_cited_response(
    rephrased_query=query,
    set_of_reference_query=unmodified_set_of_reference_material
)

if cited_response:
    print("Cited response captured...")
    print(cited_response)
else:
    print("Failed to generate cited response.")
        



Cited response captured...
{'id': 'chatcmpl-AX6YqslEI7KuvC5CJbFzQcdu5lkD5', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'You might consider purchasing life insurance from Southern Farm Bureau for several compelling reasons. First, they provide financial security, serving as a safety net for families during unexpected circumstances[1]. Their offerings include competitive premiums, which can make their policies more affordable compared to other insurers[1][2]. Southern Farm Bureau also allows for customizable policies tailored to meet individual needs and financial goals, providing flexibility in coverage options[1][3]. Furthermore, they are known for their strong reputation and history in the insurance industry, which can be reassuring to policyholders seeking a trustworthy provider[1]. Additionally, Southern Farm Bureau emphasizes quality customer service and personalized support, ensuring policyholders receive attentive and supportive ser

In [14]:
cited_response['improvement_method'] = 0

writer_citer_model_responses.append(cited_response)
writer_citer_model_responses

[{'id': 'chatcmpl-AX6YqslEI7KuvC5CJbFzQcdu5lkD5',
  'choices': [{'finish_reason': 'stop',
    'index': 0,
    'logprobs': None,
    'message': {'content': 'You might consider purchasing life insurance from Southern Farm Bureau for several compelling reasons. First, they provide financial security, serving as a safety net for families during unexpected circumstances[1]. Their offerings include competitive premiums, which can make their policies more affordable compared to other insurers[1][2]. Southern Farm Bureau also allows for customizable policies tailored to meet individual needs and financial goals, providing flexibility in coverage options[1][3]. Furthermore, they are known for their strong reputation and history in the insurance industry, which can be reassuring to policyholders seeking a trustworthy provider[1]. Additionally, Southern Farm Bureau emphasizes quality customer service and personalized support, ensuring policyholders receive attentive and supportive service from th

In [15]:
# Generate a cited response
cited_response = writer_citer.generate_cited_response(
    rephrased_query=query,
    set_of_reference_query=new_set_of_reference_material
)

if cited_response:
    print("Cited Response:")
    print(cited_response)
else:
    print("Failed to generate cited response.")




Cited Response:
{'id': 'chatcmpl-AX6Z0gSxYA7JskRttrUzMbN4WZUx9', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': "There are several compelling reasons to consider purchasing life insurance from Southern Farm Bureau. Firstly, the company provides financial security for policyholders, ensuring that families have a safety net during unexpected circumstances [1]. Southern Farm Bureau is well-regarded for offering competitive rates and customizable policies that can be tailored to fit individual needs and financial objectives [1]. \n\nThe company was founded in 1946 with a focus on serving southern farmers and their families, which underscores its commitment to community and family values [2]. Southern Farm Bureau offers a broad array of life insurance products, including both permanent and term life insurance, which are designed to suit varied life stages and financial plans [2]. Additionally, they provide financial planning services such as estat

In [16]:
cited_response['improvement_method'] = 1

writer_citer_model_responses.append(cited_response)
writer_citer_model_responses

[{'id': 'chatcmpl-AX6YqslEI7KuvC5CJbFzQcdu5lkD5',
  'choices': [{'finish_reason': 'stop',
    'index': 0,
    'logprobs': None,
    'message': {'content': 'You might consider purchasing life insurance from Southern Farm Bureau for several compelling reasons. First, they provide financial security, serving as a safety net for families during unexpected circumstances[1]. Their offerings include competitive premiums, which can make their policies more affordable compared to other insurers[1][2]. Southern Farm Bureau also allows for customizable policies tailored to meet individual needs and financial goals, providing flexibility in coverage options[1][3]. Furthermore, they are known for their strong reputation and history in the insurance industry, which can be reassuring to policyholders seeking a trustworthy provider[1]. Additionally, Southern Farm Bureau emphasizes quality customer service and personalized support, ensuring policyholders receive attentive and supportive service from th

In [17]:
for item in writer_citer_model_responses:
    print(item['choices'][0]['message']['content'])
    print("---")

You might consider purchasing life insurance from Southern Farm Bureau for several compelling reasons. First, they provide financial security, serving as a safety net for families during unexpected circumstances[1]. Their offerings include competitive premiums, which can make their policies more affordable compared to other insurers[1][2]. Southern Farm Bureau also allows for customizable policies tailored to meet individual needs and financial goals, providing flexibility in coverage options[1][3]. Furthermore, they are known for their strong reputation and history in the insurance industry, which can be reassuring to policyholders seeking a trustworthy provider[1]. Additionally, Southern Farm Bureau emphasizes quality customer service and personalized support, ensuring policyholders receive attentive and supportive service from their agents[1][2].
---
There are several compelling reasons to consider purchasing life insurance from Southern Farm Bureau. Firstly, the company provides fi

In [18]:
# Save to disk
storage_manager.save_to_folder("WriterCiterModelResponses", writer_citer_model_responses, f"{trial_number}.json")

Data saved to ../data/WriterCiterModelResponses/4.json
